# Use case: choose an airplane company
This use case applies the *analytic hierarchy process* (AHP) to choose between three airline companies **Crazy price flight**, **Oh boy it's cheap air** (OBIC air) and the **National air** for making national flights. The three candidates or *alternatives* are thus:

* *crazy air flight*
* *OBIC air*
* *National air*

The hierarchy of criteria is described below:
![Hierarchy_of_criteria](hierarchy_criteria.png)

The **covering criteria** are:

* Snack service
* Crew members
* Reliability
* Ticket price
* Price for additional services
* Price advantage with the company's partners

## General methodology

The first step of the AHP is to is carry out cross-comparison of the top-level criteria and their sub-criteria against the main objective. The comparison is carried out with the *fundamental scale of absolute numbers* given hereafter. The result of a comparison is measured with the *intensity of importance* based on the series of *odd integers* from 1 to 9. Even integers could be used for introducing a more refined assessment. However, restricting to the *odd integer* scale should be suitable in most cases. A comparison is stated as:

<p style="text-align: center;"> Intensity(A,B) = *Element A* compared to *Element B* </p>

The **reciprocal comparison** is scored with the inverse value of intensity:

<p style="text-align: center;"> Intensity(B,A) = 1/Intensity(A,B) </p>

---

<p style="text-align: center; font-weight: bold; text-decoration: underline"> Fundamental scale of absolute numbers </p>

|Intensity of importance|Definition                |Explanation                                                  |
|-----------------------|--------------------------|-------------------------------------------------------------|
|1                      |Equal importance          |The two compared elements **contribute equally** to the      |
|                       |                          |objective                                                    |
|3                      |Moderate importance       |Experience and judgment slightly favor *Element A* over      |
|                       |                          |*Element B*                                                  |
|5                      |Strong importance         |Experience and judgment strongly favor *Element A* over      |
|                       |                          |*Element B*                                                  |
|7                      |Very strong or demons-    |*Element A* is favored very strongly over *Element B*; its   |
|                       |trated importance         |dominance is demonstrated in practice                        |
|9                      |Extremely strong          |The evidence favouring *Element A* over *Element B* is of the|
|                       |importance                |**highest possible** order of affirmation                    |

---

When considering a cross-comparison of $n$ criteria, the number of independant comparisons to carry out is:

$${n \choose 2} = {n! \over (n-2)!2!} = {n(n-1) \over 2}$$

Dependant comparisons are either the comparison of an element with itself which always evaluates to 1. and reciprocal comparisons which values are inverse.

The whole set of cross-comparisons constitutes the **comparison matrix** (or *judgment matrix*) which component inversion property by transposition makes them elements of the set of **reciprocal matrices**.

<p style="font-weight: bold; text-decoration: underline;">Consistency</p> 
The *independance* of comparisons is not related to the notion of consistency. Giving a value to a comparison between elements $X$ and $Y$ and another one for the comparison between elements $Y$ and $Z$ is equivalent to state that:

<p style="text-align: center;"> $X=\alpha Y$ and $Y=\beta Z$ </p>

which implies that $X=\alpha\beta Z$. Consequently, the two comparisons are **inconsistent** if ${X\over Z}$ is significantly different from $\alpha\beta$.

<p style="font-weight: bold; text-decoration: underline;">Priorities</p>
When a cross-comparison has been carried out, *absolute priorites* can be calculated from the comparison matrix. These values are absolute because they are normalized so that their sum is equal to 1.

<p style="font-weight: bold; text-decoration: underline;">Hierarchy and covering criteria</p>
The priorities of all criteria are hierarchically weighted by the priority of their parent criterion. The *covering criteria* are the criteria at the leaves of the hierarchical tree:

* Snack onboard
* Crew members' service onboard
* Reliability of the flights' schedule
* Ticket price
* Price of additional services
* Discounts with the company's partners

Because of the hierarchical weighting their sum equals to 1.

<p style="font-weight: bold; text-decoration: underline;">Cross-comparison of alternatives</p>
The alternatives (considered solutions) are cross-compared against every *covering criterion*. The normalization of priorities implies that the sum of priorities of alternatives over covering criteria is 1.

<p style="font-weight: bold; text-decoration: underline;">Conclusion</p>
The priority of an alternative against the objective is calculated as the sum of its priorities over covering criteria.

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

def priorities(df):
    eig_val, eig_vect = np.linalg.eig(df.as_matrix())
    eig_max_ind = eig_val.argmax()
    priorities = eig_vect[:,eig_max_ind]
    # There is no imaginary part, it is mainly a type conversion
    return np.real(priorities / priorities.sum())

def inconsistency(df):
    eig_val, eig_vect = np.linalg.eig(df.as_matrix())
    n = df.shape[0]
    # consistency index
    ind_max = eig_val.argmax()
    index = (np.real(eig_val[ind_max]) - n) / (n - 1)
    # priority vector
    w = eig_vect[:,ind_max]
    w = np.real(w / w.sum())
    D = np.diag(w)
    E = np.dot(np.linalg.solve(D,df), D)
    return index, E

normalized_priorities = dict()
def update_norm_priorities(w_df):
    cnames = w_df.index.tolist()
    values = w_df.as_matrix()[:,0].tolist()
    normalized_priorities.update(dict([pair for pair in zip(cnames,values)]))
    
alternative_priorities = pd.DataFrame(np.zeros((3,)), index = ['Crazy price fl.', 'OBIC', 'National air'], 
                                      columns = ('priorities',))

## Cross-comparison of top level criteria

The 3 criteria to compare are:

* Service on board: quality of services onboard
* Reliability: flight delays or cancellation, lost bagages
* Price

A cross-comparison of 3 criteria involves 3 independant comparisons. Here is my *personal opinion*:

* *Reliability* has an *extreme importance* over *Service on board*
* *Prices* have a *very strong importance* over *Service on board*
* *Reliability* has a *slight importance* over *Prices*

These comparisons are assessed with **fundamental scale of absolute numbers**. The comparison matrix below is established from these values. The values must be read as: row-label compares to column-label:

In [2]:
labels = ['Service', 'Reliability', 'Price']
comp_mat = np.array([[1,1/9,1/7],[9,1,3],[7,1/3,1]])
comp_df = pd.DataFrame(comp_mat, index = labels, columns=labels)
display(comp_df)

,Service,Reliability,Price
Service,1.0,0.111111,0.142857
Reliability,9.0,1.000000,3.000000
Price,7.0,0.333333,1.000000


Now, let's check the *consistency* of this set of cross-comparisons:

In [3]:
ind, mat_E = inconsistency(comp_df)
print('The consistency index is:',ind,'\n')
print('The consistency deviation matrix:')
print(mat_E-1,'\n')

The consistency index is: 0.0401499218806 

The consistency deviation matrix:
[[ 0.          0.3263524  -0.24605256]
 [-0.24605256  0.          0.3263524 ]
 [ 0.3263524  -0.24605256  0.        ]] 



The statistically maximal admissible *consistency index* is 0.10 which is significantly higher than 0.040.

Eventually the **normalized priorities** are:

In [4]:
w=pd.DataFrame(priorities(comp_df), index = labels, columns = ('priorities',))
display(w.round(3))
update_norm_priorities(w)

,priorities
Service,0.055
Reliability,0.655
Price,0.290


## Comparison of level-2 sub-criteria of *Service onboard*
There are 2 criteria to compare:

* Snack service
* Crew members: are they kind? Reactive?

A cross-comparison of 2 criteria involves only 1 independant comparison. Here is my personal opinion:

* The behavior of crew members is slightly more important than the snack service onboard

In [5]:
labels = ['Snack', 'Crew members']
comp_mat = np.array([[1.,1/3],[3,1]])
comp_df = pd.DataFrame(comp_mat, index = labels, columns=labels)
display(comp_df)
ind, mat_E = inconsistency(comp_df)
print('The consistency index is:',ind,'\n')
print('The consistency deviation matrix:')
print(mat_E-1)

,Snack,Crew members
Snack,1.0,0.333333
Crew members,3.0,1.000000


The consistency index is: 0.0 

The consistency deviation matrix:
[[ 0.  0.]
 [ 0.  0.]]


A 2x2 comparison matrix is necessarily consistent. The normalized priorities are:

In [6]:
w=pd.DataFrame(priorities(comp_df), index = labels, columns = ('priorities',))
display(w.round(3))
update_norm_priorities(w)

,priorities
Snack,0.25
Crew members,0.75


## Comparison of level-2 sub-criteria of *Prices*
There are 2 criteria to compare:

* Ticket prices
* Additional services: additional checked luggages, sport equipement and special luggages
* Reductions with the company's partners: car rental, hotel reservation

A cross-comparison of 3 criteria involves 3 independant comparisons. Here is my personal opinion:

* Ticket price has a very strong importance compared to the price of additional services
* Ticket price has an extreme importance compared to reductions with the company's partners
* The price of additional services is slightly more important than reductions with the company's partners

In [7]:
labels = ['Ticket', 'Add. services', 'Reductions']
comp_mat = np.array([[1,7,9],[1/7,1,3],[1/9,1/3,1]])
comp_df = pd.DataFrame(comp_mat, index = labels, columns=labels)
display(comp_df)
ind, mat_E = inconsistency(comp_df)
print('The consistency index is:',ind,'\n')
print('The consistency deviation matrix:')
print(mat_E-1,'\n')

,Ticket,Add. services,Reductions
Ticket,1.000000,7.000000,9.0
Add. services,0.142857,1.000000,3.0
Reductions,0.111111,0.333333,1.0


The consistency index is: 0.0401499218806 

The consistency deviation matrix:
[[ 0.          0.3263524  -0.24605256]
 [-0.24605256  0.          0.3263524 ]
 [ 0.3263524  -0.24605256  0.        ]] 



The statistically maximal admissible consistency index is 0.10 which is significantly higher than 0.040.

The normalized priorities are:

In [8]:
w=pd.DataFrame(priorities(comp_df), index = labels, columns = ('priorities',))
display(w.round(3))
update_norm_priorities(w)

,priorities
Ticket,0.785
Add. services,0.149
Reductions,0.066


## Weighting of criteria
Here are the priorities of criteria before they are normalized.

In [9]:
display(normalized_priorities)

{'Add. services': 0.14881506992909246,
 'Crew members': 0.75,
 'Price': 0.2897441098717415,
 'Reductions': 0.0657937418494401,
 'Reliability': 0.6553554906601311,
 'Service': 0.054900399468127435,
 'Snack': 0.25,
 'Ticket': 0.7853911882214674}

Now, let weight the priorities hierarchically:

In [10]:
for k in ('Crew members','Snack'):
    normalized_priorities[k] *= normalized_priorities['Service']
for k in ('Ticket','Add. services','Reductions'):
    normalized_priorities[k] *= normalized_priorities['Price']
display(normalized_priorities)

{'Add. services': 0.04311828997210586,
 'Crew members': 0.041175299601095575,
 'Price': 0.2897441098717415,
 'Reductions': 0.01906334916729717,
 'Reliability': 0.6553554906601311,
 'Service': 0.054900399468127435,
 'Snack': 0.013725099867031859,
 'Ticket': 0.22756247073233843}

The covering criteria are listed below. Their weighted priorities must sum to 1.

* Snack onboard
* Crew members' service onboard
* Reliability of the flights' schedule
* Ticket price
* Price of additional services
* Discounts with the company's partners


In [11]:
covering_criteria = ('Crew members','Snack','Reliability','Ticket','Add. services','Reductions')
print('Sum of priorities of covering criteria:')
print(np.array([normalized_priorities[cov] for cov in covering_criteria]).sum())

Sum of priorities of covering criteria:
1.0


## Cross-comparison of alternatives against covering criteria

### Alternatives against *Snack onboard*

In [12]:
labels = ['Crazy price fl.', 'OBIC', 'National air']
comp_mat = np.array([[1,5,1/3],[1/5,1,1/7],[3,7,1]])
comp_df = pd.DataFrame(comp_mat, index = labels, columns=labels)
display(comp_df)
ind, mat_E = inconsistency(comp_df)
print('The consistency index is:',ind,'\n')
print('The consistency deviation matrix:')
print(mat_E-1)

,Crazy price fl.,OBIC,National air
Crazy price fl.,1.0,5.0,0.333333
OBIC,0.2,1.0,0.142857
National air,3.0,7.0,1.000000


The consistency index is: 0.0324437899364 

The consistency deviation matrix:
[[ 0.          0.28923199 -0.22434441]
 [-0.22434441  0.          0.28923199]
 [ 0.28923199 -0.22434441  0.        ]]


The statistically maximal admissible consistency index is 0.10 which is significantly higher than 0.032.

The normalized priorities are:

In [13]:
alt_w = pd.DataFrame(priorities(comp_df), index = labels, columns = ('priorities',))
display(alt_w.round(3))
alternative_priorities += alt_w * normalized_priorities['Snack']

,priorities
Crazy price fl.,0.279
OBIC,0.072
National air,0.649


### Alternatives against *Crew members*

In [14]:
labels = ['Crazy price fl.', 'OBIC', 'National air']
comp_mat = np.array([[1,3,1],[1/3,1,1/3],[1,3,1]])
comp_df = pd.DataFrame(comp_mat, index = labels, columns=labels)
display(comp_df)
ind, mat_E = inconsistency(comp_df)
print('The consistency index is:',ind,'\n')
print('The consistency deviation matrix:')
print(mat_E-1)

,Crazy price fl.,OBIC,National air
Crazy price fl.,1.000000,3.0,1.000000
OBIC,0.333333,1.0,0.333333
National air,1.000000,3.0,1.000000


The consistency index is: 0.0 

The consistency deviation matrix:
[[  0.00000000e+00   2.22044605e-16   2.22044605e-16]
 [ -2.22044605e-16   0.00000000e+00   0.00000000e+00]
 [ -2.22044605e-16   0.00000000e+00   0.00000000e+00]]


The consistency index is 0., the reciprocal matrix is consistent. The normalized priorities are:

In [15]:
alt_w = pd.DataFrame(priorities(comp_df), index = labels, columns = ('priorities',))
display(alt_w.round(3))
alternative_priorities += alt_w * normalized_priorities['Crew members']

,priorities
Crazy price fl.,0.429
OBIC,0.143
National air,0.429


### Alternatives against *Reliability*

In [16]:
labels = ['Crazy price fl.', 'OBIC', 'National air']
comp_mat = np.array([[1,3,1/2],[1/3,1,1/4],[2,4,1]])
comp_df = pd.DataFrame(comp_mat, index = labels, columns=labels)
display(comp_df)
ind, mat_E = inconsistency(comp_df)
print('The consistency index is:',ind,'\n')
print('The consistency deviation matrix:')
print(mat_E-1)

,Crazy price fl.,OBIC,National air
Crazy price fl.,1.000000,3.0,0.50
OBIC,0.333333,1.0,0.25
National air,2.000000,4.0,1.00


The consistency index is: 0.00914735364481 

The consistency deviation matrix:
[[  0.00000000e+00   1.44714243e-01  -1.26419535e-01]
 [ -1.26419535e-01   0.00000000e+00   1.44714243e-01]
 [  1.44714243e-01  -1.26419535e-01  -1.11022302e-16]]


The statistically maximal admissible consistency index is 0.10 which is significantly higher than 0.009. The normalized priorities are:

In [17]:
alt_w = pd.DataFrame(priorities(comp_df), index = labels, columns = ('priorities',))
display(alt_w.round(3))
alternative_priorities += alt_w * normalized_priorities['Reliability']

,priorities
Crazy price fl.,0.320
OBIC,0.122
National air,0.558


### Alternatives against *Ticket price*

In [18]:
labels = ['Crazy price fl.', 'OBIC', 'National air']
comp_mat = np.array([[1,1/3,5],[3,1,7],[1/5,1/7,1]])
comp_df = pd.DataFrame(comp_mat, index = labels, columns=labels)
display(comp_df)
ind, mat_E = inconsistency(comp_df)
print('The consistency index is:',ind,'\n')
print('The consistency deviation matrix:')
print(mat_E-1)

,Crazy price fl.,OBIC,National air
Crazy price fl.,1.0,0.333333,5.0
OBIC,3.0,1.000000,7.0
National air,0.2,0.142857,1.0


The consistency index is: 0.0324437899364 

The consistency deviation matrix:
[[ 0.         -0.22434441  0.28923199]
 [ 0.28923199  0.         -0.22434441]
 [-0.22434441  0.28923199  0.        ]]


The statistically maximal admissible consistency index is 0.10 which is significantly higher than 0.032. The normalized priorities are:

In [19]:
alt_w = pd.DataFrame(priorities(comp_df), index = labels, columns = ('priorities',))
display(alt_w.round(3))
alternative_priorities += alt_w * normalized_priorities['Ticket']

,priorities
Crazy price fl.,0.279
OBIC,0.649
National air,0.072


### Alternatives against *Price of additional services*

In [20]:
labels = ['Crazy price fl.', 'OBIC', 'National air']
comp_mat = np.array([[1,3,1/7],[1/3,1,1/8],[7,8,1]])
comp_df = pd.DataFrame(comp_mat, index = labels, columns=labels)
display(comp_df)
ind, mat_E = inconsistency(comp_df)
print('The consistency index is:',ind,'\n')
print('The consistency deviation matrix:')
print(mat_E-1)

,Crazy price fl.,OBIC,National air
Crazy price fl.,1.000000,3.0,0.142857
OBIC,0.333333,1.0,0.125000
National air,7.000000,8.0,1.000000


The consistency index is: 0.0521911684296 

The consistency deviation matrix:
[[  0.00000000e+00   3.79462088e-01  -2.75079751e-01]
 [ -2.75079751e-01   0.00000000e+00   3.79462088e-01]
 [  3.79462088e-01  -2.75079751e-01  -1.11022302e-16]]


The statistically maximal admissible consistency index is 0.10 which is significantly higher than 0.052. The normalized priorities are:

In [21]:
alt_w = pd.DataFrame(priorities(comp_df), index = labels, columns = ('priorities',))
display(alt_w.round(3))
alternative_priorities += alt_w * normalized_priorities['Add. services']

,priorities
Crazy price fl.,0.153
OBIC,0.070
National air,0.777


### Alternatives against *Discount with partners*

In [22]:
labels = ['Crazy price fl.', 'OBIC', 'National air']
comp_mat = np.array([[1,2,1],[1/2,1,1/2],[1,2,1]])
comp_df = pd.DataFrame(comp_mat, index = labels, columns=labels)
display(comp_df)
ind, mat_E = inconsistency(comp_df)
print('The consistency index is:',ind,'\n')
print('The consistency deviation matrix:')
print(mat_E-1)

,Crazy price fl.,OBIC,National air
Crazy price fl.,1.0,2.0,1.0
OBIC,0.5,1.0,0.5
National air,1.0,2.0,1.0


The consistency index is: 0.0 

The consistency deviation matrix:
[[  0.00000000e+00  -3.33066907e-16   0.00000000e+00]
 [  4.44089210e-16   0.00000000e+00   4.44089210e-16]
 [  0.00000000e+00  -3.33066907e-16   0.00000000e+00]]


The consistency index is 0., the reciprocal matrix is consistent. The normalized priorities are:

In [23]:
alt_w = pd.DataFrame(priorities(comp_df), index = labels, columns = ('priorities',))
display(alt_w.round(3))
alternative_priorities += alt_w * normalized_priorities['Reductions']

,priorities
Crazy price fl.,0.4
OBIC,0.2
National air,0.4


The priorities of alternatives are multiplied with the priorities of the *covering criteria* they are compared against. They are eventually summed which gives their **priority against the main objective**.

It is checked that these priorities sum to 1.

In [24]:
display(alternative_priorities.round(3))
print('Sum of priorities = {0:f}'.format(alternative_priorities.as_matrix().sum()))

,priorities
Crazy price fl.,0.309
OBIC,0.241
National air,0.450


Sum of priorities = 1.000000


## CONCLUSION
The winner is **National Air** with a score of 45.0%, followed by **Crazy price flights** with a score of 30.9%.  **Oh Boy It's Cheap Air** with a score of 24.1% is in last position.